In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import os

In [ ]:
# Load the features dataset
file_path = './features_dataset.csv'
if not os.path.exists(file_path):
    raise FileNotFoundError(f"{file_path} not found. Ensure the file exists in the correct directory.")

features_df = pd.read_csv(file_path)

In [ ]:
# Define mood labels
def assign_mood(row):
    if row['valence'] > 0.6 and row['energy'] > 0.7:
        return 'energetic'
    elif row['valence'] < 0.4 and row['energy'] < 0.5:
        return 'sad'
    elif row['valence'] > 0.6:
        return 'happy'
    else:
        return 'calm'

features_df['mood'] = features_df.apply(assign_mood, axis=1)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
features_df['mood_encoded'] = label_encoder.fit_transform(features_df['mood'])


In [7]:
# Save the label encoder for testing later
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
# Select features and target
# Drop unnecessary columns
X = features_df.drop(columns=['mood', 'mood_encoded'], errors='ignore')  # Features
y = features_df['mood_encoded']  # Target

In [ ]:
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Save the scaler for testing later
import joblib
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [ ]:
# DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Our NN
class MoodPredictorNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MoodPredictorNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  
        return x


# class MoodPredictorNN(nn.Module):
#     def __init__(self, input_size, num_classes): # Input size - 12 features for each song
#         super(MoodPredictorNN, self).__init__()
#         self.fc1 = nn.Linear(input_size, 64)  # Input → Hidden Layer 1
#         self.fc2 = nn.Linear(64, 32)         # Hidden Layer 1 → Hidden Layer 2
#         self.fc3 = nn.Linear(32, num_classes)  # Hidden Layer 2 → Output Layer

#     def forward(self, x):
#         x = F.relu(self.fc1(x))  # ReLU Activation for Layer #1
#         x = F.relu(self.fc2(x))  # ReLU Activation for Layer #2
#         x = self.fc3(x)          # Output logits 
#         return x

In [ ]:
# Initialize model
input_size = X_train_tensor.shape[1]
num_classes = len(label_encoder.classes_)
model = MoodPredictorNN(input_size, num_classes)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)    # Forward pass
        loss = criterion(outputs, y_batch)  # Calc loss
        
        # Backward pass
        optimizer.zero_grad()   # Clear gradients
        loss.backward()     # Backpropagation
        optimizer.step()    # Update weights
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/20, Loss: 0.1429
Epoch 2/20, Loss: 0.0418
Epoch 3/20, Loss: 0.0323
Epoch 4/20, Loss: 0.0277
Epoch 5/20, Loss: 0.0255
Epoch 6/20, Loss: 0.0247
Epoch 7/20, Loss: 0.0227
Epoch 8/20, Loss: 0.0221
Epoch 9/20, Loss: 0.0210
Epoch 10/20, Loss: 0.0205
Epoch 11/20, Loss: 0.0206
Epoch 12/20, Loss: 0.0202
Epoch 13/20, Loss: 0.0195
Epoch 14/20, Loss: 0.0181
Epoch 15/20, Loss: 0.0187
Epoch 16/20, Loss: 0.0185
Epoch 17/20, Loss: 0.0180
Epoch 18/20, Loss: 0.0178
Epoch 19/20, Loss: 0.0180
Epoch 20/20, Loss: 0.0174


In [ ]:
# Model Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 98.68%


In [ ]:
# Save the model
torch.save(model.state_dict(), 'mood_prediction_model.pth')
print("Model saved as mood_prediction_model.pth")

Model saved as mood_prediction_model.pth
